In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
from matplotlib.ticker import EngFormatter,MaxNLocator
import datetime
#import seaborn as sns
#sns.set()

# Get the filepath list

In [ ]:
# time range
_ = md.date2num(datetime.datetime.now())
xlimit = [_-2,_]

# Bridge for still capacitance
dirBridge = r'C:\Impedance Bridge\Data'
pathListB = []
for i in sorted(os.listdir(dirBridge))[::-1]:
    _ = md.datestr2num(i.split(' ')[0])#start time of the file
    if _<xlimit[1]:
        print(i)
        pathListB.append(os.path.join(dirBridge,i))
        if _<xlimit[0]:
            break
            
# temperatures and pressures
dirAVS = r'C:\avs-47'
avslist = os.listdir(dirAVS)
pathListT = []
for i in sorted([x for x in avslist if 'LogAVS' in x and x!='LogAVS__zBridge_2014-09-04-11-39-58_1.dat'])[::-1]:
    _ = i.replace('.dat','').split('_')[-1]#something like '2014-09-04-11-39-58'
    _ = '%s-%s-%s %s:%s:%s'%tuple(_.split('-'))
    _ = md.datestr2num(_)#start time of the file
    if _<xlimit[1]:
        print(i)
        pathListT.append(os.path.join(dirAVS,i))
        if _<xlimit[0]:
            break
            
pathListP = []
for i in sorted([x for x in avslist if 'logFP' in x])[::-1]:
    _ = i.replace('.dat','').split('_')[-1]#something like '2014-09-04-11-39-58'
    _ = '%s-%s-%s %s:%s:%s'%tuple(_.split('-'))
    _ = md.datestr2num(_)#start time of the file
    if _<xlimit[1]:
        print(i)
        pathListP.append(os.path.join(dirAVS,i))
        if _<xlimit[0]:
            break


# Get the capacitance and show the average over the last 30 pts

In [ ]:
_ = ['Time','Impedance','Calculated value','Frequency','Excitation','Chno','Measure type','Imp type']
headBridge = dict(zip(_,range(len(_))))
dBridge = None
for i in pathListB:
    _ = np.loadtxt(i,delimiter='\t',converters={0:lambda s: md.datestr2num(s.decode()),7:lambda s: int(s[0])})
    dBridge = _ if dBridge is None else np.append(_,dBridge,axis=0)
_ = -30
print('Average from: %s to %s, dt = %s'%(md.num2date(dBridge[_,0]),md.num2date(dBridge[-1,0]),md.num2date(dBridge[-1,0])-md.num2date(dBridge[_,0])))
print('mean: %s, std: %s'%(np.mean(dBridge[_:,1])*1e12,np.std(dBridge[_:,1]*1e12)))

# Get the temperatures and the pressures

In [ ]:
_ = ['Date', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'Flow', 'MG0', 'MG1', 'MG2', 'MG3', 'MG4', 'MG5', 'PT on', 'W_in', 'W_out', 'He', 'Oil', 'P_low', 'P_high', 'PT current', 'State0', 'State1', 'State2']
headFP = dict(zip(_,range(len(_))))
_ = ['Date1', 'Date2', 'R0', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'I0', 'I1', 'I2', 'I3']
headTC = dict(zip(_,range(len(_))))
dTC = None
for i in pathListT:
    try:
        _ = np.loadtxt(i,delimiter='\t',converters = {0:lambda s: md.datestr2num(s.decode())})#sometimes the head is missing
    except:
        _ = np.loadtxt(i,delimiter='\t',converters = {0:lambda s: md.datestr2num(s.decode())},skiprows=4)
    dTC = _ if dTC is None else np.append(_,dTC,axis=0)
dFP = None
for i in pathListP:
    _ = np.loadtxt(i,delimiter='\t',converters = {0:lambda s: md.datestr2num(s.decode())})
    dFP = _ if dFP is None else np.append(_,dFP,axis=0)

# Plot the data

In [ ]:
def getXYL(key2,label):
    key = key2.split('_')[-1] if key2.startswith('MA') else key2        
    if key in headFP:
        x = dFP[:,0]
        y = dFP[:,headFP[key]]
    elif key in headTC:
        x = dTC[:,0]
        y = dTC[:,headTC[key]]/1000.
    else:
        x = dBridge[:,0]
        y = dBridge[:,headBridge[key]]
    if key2.startswith('MA'):
        _ = int(key2.split('_')[0][2:])
        x = x[_:-_]
        _ = _ * 2 + 1
        y = np.convolve(y,np.ones(_)/_,'valid')
        label = 'smooth\n%s pts'%(_)

    return x,y,label

In [ ]:
cv = [{'Impedance':'Cstill','MA15_Impedance':'Cstill'},
      {'T0':'3K','T1':'still','T2':'cold','T5':'magnet','limit':(0,5)},
      {'P5':'P5\n(mbar)','P6':'He4','P7':'He3','limit':(-20,1420),'ticks':[0,200,400,600,800,1000,1200,1400]},
     ]#curves
cv2 = [{},
       {'T3':'MC','limit':(0.042,0.045)},
       {'Flow':'Flow','limit':(0,100)}
      ]
fig, ax = plt.subplots(len(cv),1,figsize=(10, 8),sharex=True)
fig.subplots_adjust(hspace=0.05)
ax2 = [i.twinx() for i in ax]

cv = [*cv, *cv2]
ax = [*ax, *ax2]

xfmt = md.DateFormatter('%m-%d %H:%M')
yfmt = EngFormatter()

for i,j,k in zip(cv,ax,range(len(cv))):
    for ii in i:
        if ii == 'limit':
            j.set_ylim(*i[ii])
        elif ii == 'ticks':
            j.set_yticks(i[ii])
        else:
            x,y,lbl = getXYL(ii,i[ii])
            if k>=len(cv)/2:
                for iii in range(len(cv[k-int(len(cv)/2)])):
                    j._get_lines.get_next_color()
            j.plot(x,y,label=lbl)
    if i:
        j.yaxis.set_major_formatter(yfmt)
        if k<len(cv)/2:
            j.legend(bbox_to_anchor=(1.25, 1),loc='upper right')
        else:
            j.legend(bbox_to_anchor=(1.25, 0.5),loc='upper right')
        j.xaxis.set_major_formatter(xfmt)
        j.xaxis.set_major_locator(md.HourLocator(interval=4))
        plt.sca(j)
        plt.xticks(rotation=25)
    else:
        j.set_yticks([])

plt.subplots_adjust(bottom=0.1,right=0.8,left=0.1)

#ax3.yaxis.set_major_formatter(yfmt)
#ax3.legend(bbox_to_anchor=(1.25, 0.5),loc='upper right')
#ax3.tick_params(which='both',axis='y', colors='red')
#ax3.set_xlim(*xlimit)

j.set_xlim(*xlimit)
j.set_xlabel('time')
plt.savefig(datetime.datetime.now().strftime("%Y%m%d_%H.png"),dpi=300)

In [ ]:
#plt.savefig('a.png')